In [ ]:
!pip install 

In [3]:
import requests
from urllib.parse import urlparse
from io import BytesIO
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from dotenv import load_dotenv

import os, time, uuid
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials

load_dotenv()

True

In [4]:
CONFIG = {
    'TRAINING_ENDPOINT' : os.getenv('OBJECT_DETECTION_TRAINING_ENDPOINT'), 
    'TRAINING_KEY' : os.getenv('OBJECT_DETECTION_TRAINING_KEY'),
    'TRAINING_RESOURCE_ID' : os.getenv('OBJECT_DETECTION_TRAINING_RESOURCE_ID'),

    'PREDICTION_ENDPOINT' : os.getenv('OBJECT_DETECTION_PREDICTION_ENDPOINT'),
    'PREDICTION_KEY' : os.getenv('OBJECT_DETECTION_PREDICTION_KEY'),
    'PREDICTION_RESOURCE_ID' : os.getenv('OBJECT_DETECTION_PREDICTION_RESOURCE_ID')
}

In [5]:
def show_image_in_cell(img_url):
    response = requests.get(img_url)
    img = Image.open(BytesIO(response.content))
    plt.figure(figsize=(20,10))
    plt.imshow(img)
    plt.show()

In [6]:
training_credentials = ApiKeyCredentials(in_headers={"Training-key": CONFIG['TRAINING_KEY']})
trainer = CustomVisionTrainingClient(CONFIG['TRAINING_ENDPOINT'], training_credentials)
trainer.api_version

'3.4-preview'

In [7]:
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": CONFIG['PREDICTION_KEY']})
predictor = CustomVisionPredictionClient(CONFIG['PREDICTION_ENDPOINT'], prediction_credentials)
predictor.api_version

'3.1'

### Model Prediction

In [8]:
publish_iteration_name = "udacity-object-detection"
project_id = "79bde101-e838-4b1c-8264-3bf2779da1b4"
iteration_id = "a85a8cb3-56bd-4566-965f-bbc91967f58f"
trainer.publish_iteration(project_id, iteration_id, publish_iteration_name, CONFIG['TRAINING_RESOURCE_ID'])
print ("Done!")

Done!


In [13]:
local_image_path = 'data\lighter_test_images'
def perform_prediction(image_file_name):
    with open(os.path.join (local_image_path,  image_file_name), "rb") as image_contents:
        results = predictor.detect_image(project_id, publish_iteration_name, image_contents.read())
        # Display the results.
        for prediction in results.predictions:
            print("\t" + prediction.tag_name +
                  ": {0:.2f}%".format(prediction.probability * 100))

In [15]:
file_name = "lighter_test_set_1of5.jpg"
perform_prediction(file_name)
## Checking the Image
with open(os.path.join (local_image_path, file_name), 'rb') as img_code:
    img_view_ready = Image.open(img_code)
    plt.figure()
    plt.imshow(img_view_ready)

CustomVisionErrorException: Invalid iteration

### Exporting Model

In [16]:
platform = "TensorFlow"
flavor = "TensorFlowLite"
export_process = trainer.export_iteration(project_id, iteration_id, platform, flavor, raw=True)

In [17]:
print(export_process.output)

{'additional_properties': {}, 'platform': 'TensorFlow', 'status': 'Exporting', 'download_uri': None, 'flavor': 'TensorFlowLite', 'newer_version_available': False}


In [18]:
print(export_process.output.status)

Exporting


In [20]:
### Code snippet is from Azure SDK and Documentation
### https://docs.microsoft.com/en-us/azure/cognitive-services/custom-vision-service/export-programmatically
### This step may take long time 
while (export_process.output.status == "Exporting"):
    print ("Waiting 10 seconds...")
    time.sleep(10)
    exports = trainer.get_exports(project_id, iteration_id)
    for e in exports:
        if e.platform == export_process.output.platform and e.flavor == export_process.output.flavor:
            export = e
            break
    print("Export status is: ", export_process.output.status)

Waiting 10 seconds...
Export status is:  Exporting
Waiting 10 seconds...
Export status is:  Exporting
Waiting 10 seconds...
Export status is:  Exporting
Waiting 10 seconds...
Export status is:  Exporting
Waiting 10 seconds...
Export status is:  Exporting
Waiting 10 seconds...
Export status is:  Exporting
Waiting 10 seconds...
Export status is:  Exporting
Waiting 10 seconds...
Export status is:  Exporting
Waiting 10 seconds...
Export status is:  Exporting
Waiting 10 seconds...
Export status is:  Exporting
Waiting 10 seconds...
Export status is:  Exporting
Waiting 10 seconds...
Export status is:  Exporting
Waiting 10 seconds...
Export status is:  Exporting
Waiting 10 seconds...
Export status is:  Exporting
Waiting 10 seconds...
Export status is:  Exporting
Waiting 10 seconds...


In [ ]:
print(export_process.output.status)
print(export_process.output.download_uri)

In [ ]:
## Downloading the model from url
if export_process.output.status == "Done":
    # Ready to Download
    model_export_file = requests.get(export_process.output.download_uri)
    with open("od_model_tensorflow.zip", "wb") as file:
        file.write(model_export_file.content)